# Lennard Jones in the canonical ensemble

In this example, we will reproduce the [average energy reported in the NIST SRSW](https://mmlapps.nist.gov/srs/LJ_PURE/mc.htm) at $T^*=0.9$ and $\rho^*=0.003$ using a Monte Carlo simulation of a bulk Lennard Jones fluid ([Section IIIB of the manuscript](https://doi.org/10.1063/5.0224283)).
The simulation will be initialized with a desired number of particles in a cubic box, equilibrated while tuning the maximum trial displacement before obtaining the ensemble average energy.

To begin, Python is used to generate a text file, `script.txt`, which is then input to the ```fst``` executable using the command ```fst < script.txt```.
We recommend using Python to generate a text file, instead of manually editting a text file, for the following reasons:

 - Variables required by FEASST arguments may need to be converted from variables that the user prefers to input, and these FEASST arguments can then be generated by Python to machine precision. For example, if the user prefers to input temperature and state a temperature of $T^*=0.9$ in their manuscript, but FEASST requires $\beta=\frac{1}{k_B T}$, then $T^*=0.9$ is easier to input than $\beta=1.111111111111111$.
 - Python generation of input files may create multiple related simulations from a single source. For example, the next tutorial shows how to simulate a range of densities.
 - Python provides formatted strings and argument parsing.

In this example, we begin a simulation by defining a Python string `script`, where comments can be input for future reference, and a [MonteCarlo](../plugin/monte_carlo/doc/MonteCarlo_arguments.rst) object is created.
Then, a random number generator is given a seed.
The available arguments of [RandomMT19937](../plugin/math/doc/RandomMT19937_arguments.rst) can be found in the documentation of that object, or its base-class, [Random](../plugin/math/doc/Random_arguments.rst). 
Arguments of objects are separated by a space and are provided first as the name of the argument and then the value. In this case, `seed` is the argument name and `1572362164` is the argument value. 

In [1]:
# This and all tutorial code blocks are in Python.
# Copy/paste or use the URL to find the code.
# For example, https://pages.nist.gov/feasst/tutorial/tutorial.html is $HOME/feasst/tutorial/tutorial.ipynb . Usage: jupyter notebook tutorial.ipynb
# And https://pages.nist.gov/feasst/tutorial/launch.html is $HOME/feasst/tutorial/launch.py . Usage: python launch.py

import subprocess
    
script="""
# comments begin with the '#' symbol
MonteCarlo
RandomMT19937 seed 1572362164
"""

def run(script):
    with open('script.txt', 'w') as file: file.write(script)
    syscode = subprocess.call("../build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
    if syscode != 0:
        print('syscode', syscode)
        print('If you see "/bin/bash: line 1: ../build/bin/fst: No such file or directory" then either fst was not compiled or you need to change the path to fst in subprocess.call(...) two lines above.')
    else:
        with open('script.log', 'r') as file:
            print(file.read())
run(script)

# FEASST version: 0.25.5
# Usage: ./fst < file.txt
MonteCarlo
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164



The output of ```fst``` in ```script.log``` is displayed, and may be used to reproduce the simulation.
It includes the FEASST version as well as a confirmation of the seed.
The version shown may not match your version.

If there is a typo in the arguments, this may result in an exception which will print to the terminal. You may test this now by changing the `seed` argument to `sd`. Because the argument `sd` is not recognized by [RandomMT19937](../plugin/math/doc/RandomMT19937_arguments.rst), a nonzero system code should be returned and a verbose error should appear in your terminal.

The next step is to add a [Configuration](../plugin/configuration/doc/Configuration_arguments.rst).
In this example, a simple cubic periodic box is defined based on the number of particles and density.
Because the cubic length needs to be input to high precision, a Python format string is used to input a variable enclosed in curly brackets.

In addition, the first particle type with an index of `0` is defined by the file `lj.txt`.
Any argument value beginning with "/feasst" will have that beginning replaced with [feasst::FEASST_INSTALL_DIR](../plugin/utils/doc/definitions.rst), and thus the full path to the file `lj.txt` is displayed in `script.log` shown below.
See [Particle](../particle/README.rst) for more information about the format of the data file, which is a LAMMPS-inspired file with some major differences.

In [2]:
num_particles=500
density=0.003
script+="""
Configuration cubic_side_length {length} particle_type0 /feasst/particle/lj.txt
""".format(length=(num_particles/density)**(1./3.))
run(script)

# FEASST version: 0.25.5
# Usage: ./fst < file.txt
MonteCarlo
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_side_length 55.03212081491043 particle_type0 /home/user/feasst/particle/lj.txt  



Two [Potentials](../plugin/system/doc/Potential_arguments.rst) are introduced.
First, the pair-wise [Lennard-Jones (LJ) model](../plugin/system/doc/LennardJones_arguments.rst).
Because the domain is large compared to the potential cutoff, a [cell list](../plugin/system/doc/VisitModelCell_arguments.rst) is used.
Second, [long-range corrections](../plugin/system/doc/LongRangeCorrections_arguments.rst), which approximately account for the cut off of the LJ potential by assuming a pair-wise radial distribution function of unity, are also initialized.

In [3]:
script+="""
Potential Model LennardJones VisitModel VisitModelCell min_length max_cutoff
Potential VisitModel LongRangeCorrections
"""

Next, set [ThermoParams](../plugin/system/doc/ThermoParams_arguments.rst), such as temperature and chemical potential of each particle type.
The initial configuration will be generated with grand canonical particle additions, so a chemical potential is included, but will not contribute to canonical ensemble production simulations.

In addition, [Metropolis](../plugin/monte_carlo/doc/Metropolis_arguments.rst) acceptance criteria is utilized.

In [4]:
script+="""
ThermoParams beta {beta} chemical_potential0 -1
Metropolis
""".format(beta=1./0.9)

A [TrialTranslate](../plugin/monte_carlo/doc/TrialTranslate_arguments.rst) is then introduced which attempts to translate a random particle by a random distance which is bound in each dimension by a `tunable_param`.
This parameter may be adjusted to obtain a desired acceptance ratio, `tunable_target_acceptance`, with the help of [Tune](../plugin/steppers/doc/Tune_arguments.rst).

In [5]:
script+="""
TrialTranslate weight 1 tunable_param 2 tunable_target_acceptance 0.2
Tune
"""
run(script)

# FEASST version: 0.25.5
# Usage: ./fst < file.txt
MonteCarlo
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_side_length 55.03212081491043 particle_type0 /home/user/feasst/particle/lj.txt  
Potential Model LennardJones VisitModel VisitModelCell min_length max_cutoff  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.1111111111111112 chemical_potential0 -1  
Metropolis  
TrialTranslate tunable_param 2 tunable_target_acceptance 0.2 weight 1  
Tune  



[CheckEnergy](../plugin/steppers/doc/CheckEnergy_arguments.rst) asserts that the optimized energy calculations match the unoptimized calculations within a tolerance.
Because the unoptimized energy calculation can be expensive, this check is only performed every `trials_per_update`.

In [6]:
script+="""
CheckEnergy trials_per_update 1e4 decimal_places 8
"""

With the help of [TrialAdd](../plugin/monte_carlo/doc/TrialTranslate_arguments.rst), we can now generate an initial configuration with a [Run](../plugin/monte_carlo/doc/Run_arguments.rst) until the desired number of particles are reached. [Remove](../plugin/monte_carlo/doc/Remove_arguments.rst) is then used to revert back to the canonical ensemble.

In [7]:
script+="""
TrialAdd weight 2 particle_type 0
Run until_num_particles {num_particles}
Remove name TrialAdd
""".format(num_particles=num_particles)
run(script)

# FEASST version: 0.25.5
# Usage: ./fst < file.txt
MonteCarlo
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_side_length 55.03212081491043 particle_type0 /home/user/feasst/particle/lj.txt  
Potential Model LennardJones VisitModel VisitModelCell min_length max_cutoff  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.1111111111111112 chemical_potential0 -1  
Metropolis  
TrialTranslate tunable_param 2 tunable_target_acceptance 0.2 weight 1  
Tune  
CheckEnergy decimal_places 8 trials_per_update 1e4  
TrialAdd particle_type 0 weight 2  
Run until_num_particles 500  
Remove name TrialAdd  



The simulation is then [Run](../plugin/monte_carlo/doc/Run_arguments.rst) for a number of equilibration trials before [Remove](../plugin/monte_carlo/doc/Remove_arguments.rst) disables tuning of the maximum displacement.

In [8]:
script+="""
Run num_trials 1e5
Remove name Tune
"""

Finally, it is time to obtain the ensemble average energy from a production simulation.
First, a [Log](../plugin/steppers/doc/Log_arguments.rst) periodically outputs the instantaneous status of the trials, but it is more useful for watching the progress of a simulation than actually calculating quantities.
Instead of analyzing the Log file, [Energy](../plugin/steppers/doc/Energy_arguments.rst) may be used to more precisely compute the average energy by accumulating its value after every trial.
A trajectory in the XYZ format may also be produced using [Movie](../plugin/steppers/doc/Movie_arguments.rst).


Additional [Analyze](../plugin/monte_carlo/doc/Analyze_arguments.rst) or [Modify](../plugin/monte_carlo/doc/Modify_arguments.rst) may be added to perform some task contingent upon the number of attempted trials.

In [9]:
script+="""
Log trials_per_write 1e4 output_file lj.txt
Energy trials_per_write 1e4 output_file lj_en.txt
Movie trials_per_write 1e4 output_file lj.xyz
Run num_trials 1e6
"""
run(script)

# FEASST version: 0.25.5
# Usage: ./fst < file.txt
MonteCarlo
RandomMT19937 seed 1572362164  
# initializing random number generator with seed: 1572362164
Configuration cubic_side_length 55.03212081491043 particle_type0 /home/user/feasst/particle/lj.txt  
Potential Model LennardJones VisitModel VisitModelCell min_length max_cutoff  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.1111111111111112 chemical_potential0 -1  
Metropolis  
TrialTranslate tunable_param 2 tunable_target_acceptance 0.2 weight 1  
Tune  
CheckEnergy decimal_places 8 trials_per_update 1e4  
TrialAdd particle_type 0 weight 2  
Run until_num_particles 500  
Remove name TrialAdd  
Run num_trials 1e5  
Remove name Tune  
Log output_file lj.txt trials_per_write 1e4  
Energy output_file lj_en.txt trials_per_write 1e4  
Movie output_file lj.xyz trials_per_write 1e4  
Run num_trials 1e6  



Compare the [average energy to the NIST SRSW](https://mmlapps.nist.gov/srs/LJ_PURE/mc.htm)

In [10]:
import pandas as pd
df=pd.read_csv('lj_en.txt')
print('U_FEASST/N=', df['average'][0]/num_particles, '+/-', df['block_stdev'][0]/num_particles)
print('U_SRSW/N=', -2.9787E-02, '+/-', 3.21E-05)
import math
assert abs(-2.9787E-02-df['average'][0]/num_particles) < 3*math.sqrt(3.21E-05**2+(df['block_stdev'][0]/num_particles)**2)

U_FEASST/N= -0.030332409560752716 +/- 0.0002319803861581972
U_SRSW/N= -0.029787 +/- 3.21e-05


You should also find the `lj.xyz` trajectory file with an automatically-generated `lj.xyz.vmd` file for use with VMD (e.g., `vmd -e lj.xyz.vmd`).

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../CONTACT.rst) us. Any feedback is appreciated!